In [24]:
CONFIG_FILE = "config.json"
MODEL_WEIGHTS = "pytorch_model.bin"
VOCAB_FILE = "vocab.txt"

from transformers import BertTokenizer

TOKEN_CLASS = BertTokenizer

In [45]:
from BertSeqClassProcessData import *
from BertSeqClassGlobalVar import global_var
from BertSeqClassDataLoader import *

use_qe = True
n_chars_trim = 200

training_data_path = "../data/trials_topics_combined_all_years_qe_paper.pickle"

df = pd.read_pickle(training_data_path)
df_input = process_baseline_raw(df, use_qe, n_chars_trim, global_var)

Inside process_baseline_raw(), checking for topic_year mapping, all years present. Data will be split into training and testing later.
0    2017_18
1    2017_27
2    2017_28
3    2017_30
4     2017_7
Name: topic_year, dtype: object
{'2017_1': 0, '2017_11': 1, '2017_12': 2, '2017_13': 3, '2017_14': 4, '2017_15': 5, '2017_16': 6, '2017_17': 7, '2017_18': 8, '2017_19': 9, '2017_2': 10, '2017_20': 11, '2017_21': 12, '2017_22': 13, '2017_23': 14, '2017_24': 15, '2017_25': 16, '2017_26': 17, '2017_27': 18, '2017_28': 19, '2017_29': 20, '2017_3': 21, '2017_30': 22, '2017_4': 23, '2017_5': 24, '2017_6': 25, '2017_7': 26, '2017_8': 27, '2017_9': 28, '2018_1': 29, '2018_10': 30, '2018_11': 31, '2018_12': 32, '2018_13': 33, '2018_14': 34, '2018_15': 35, '2018_16': 36, '2018_17': 37, '2018_18': 38, '2018_19': 39, '2018_2': 40, '2018_20': 41, '2018_21': 42, '2018_22': 43, '2018_23': 44, '2018_24': 45, '2018_25': 46, '2018_26': 47, '2018_27': 48, '2018_28': 49, '2018_29': 50, '2018_3': 51, '2018_30'

In [46]:
df_input.head()

,id,year,gene,disease,brief_summary,brief_title,topic,label,qe_all,brief_t_and_s,topic_year,ty_id
0,NCT01774162,2017,CDK6 Amplification,Pancreatic cancer,Endoscopic ultrasound (EUS) is a well-establis...,EUS-guided Fine Needle Biopsy With a New Core ...,18,0,"[Pancreatic cancer, CDK6 Amplification, MCPH12...",EUS-guided Fine Needle Biopsy With a New Core ...,2017_18,8
1,NCT01774162,2017,"KRAS, TP53",Pancreatic adenocarcinoma,Endoscopic ultrasound (EUS) is a well-establis...,EUS-guided Fine Needle Biopsy With a New Core ...,27,0,"[Pancreatic adenocarcinoma, KRAS, TP53, 'C-K-R...",EUS-guided Fine Needle Biopsy With a New Core ...,2017_27,18
2,NCT01774162,2017,ERBB3,Pancreatic ductal adenocarcinoma,Endoscopic ultrasound (EUS) is a well-establis...,EUS-guided Fine Needle Biopsy With a New Core ...,28,0,"[Pancreatic ductal adenocarcinoma, ERBB3, ErbB...",EUS-guided Fine Needle Biopsy With a New Core ...,2017_28,19
3,NCT01774162,2017,"RB1, TP53, KRAS",Pancreatic adenocarcinoma,Endoscopic ultrasound (EUS) is a well-establis...,EUS-guided Fine Needle Biopsy With a New Core ...,30,0,"[Pancreatic adenocarcinoma, RB1, TP53, KRAS, O...",EUS-guided Fine Needle Biopsy With a New Core ...,2017_30,22
4,NCT01226147,2017,EGFR (L858R),Lung cancer,An open-label study to evaluate the efficacy a...,Efficacy and Safety of Tamibarotene(AM80) for ...,7,0,"[Lung cancer, EGFR (L858R), ERBB ERBB1 HER1 NI...",Efficacy and Safety of Tamibarotene(AM80) for ...,2017_7,26


In [47]:
global_var["seq_a_expansion"]

'qe_all'

In [48]:
if use_qe:
    seq_a_col = global_var["seq_a_expansion"]
else:
    seq_a_col = global_var["seq_a_baseline"]

train_year = [2017, 2018]
test_year = [2019]
topic_subset=None

print(seq_a_col)

seq_a_train, seq_b_train, \
labels_train, attribute_seq_train, doc_ids_train, \
seq_a_test, seq_b_test, \
labels_test, attribute_seq_test, doc_ids_test = split_into_train_test_set(
    df_input,
    seq_a_col,
    global_var["seq_b_baseline"],
    train_year,
    test_year,
    topic_subset=topic_subset,
    use_qe=use_qe,
    global_var=global_var
)

qe_all
df_test.shape: (12996, 12)
df_train.shape: (27207, 12)


In [49]:
seq_a_train

['Pancreatic cancer CDK6 Amplification MCPH12 PLSTIRE cyclindependent kinase 6 cell division protein kinase 6 serine threonineprotein kinase PLSTIRE Malignant tumor of pancreas Malignant tumor of pancreas disorder CA Pancreatic cancer Pancreatic cancer CA Cancer of pancreas Malignant tumour of pancreas',
 'Pancreatic adenocarcinoma KRAS TP53 CKRAS CKRAS CFC2 KRAS2A KRAS2B KRAS4A KRAS4B KRas KRas 2 KIRAS KRAS1 KRAS2 NS NS3 OES RALD RASK2 cKiras cKiras2 BCC7 BMFS5 LFS1 P53 TRP53',
 'Pancreatic ductal adenocarcinoma ERBB3 ErbB3 FERLK HER3 LCCS2 MDABF1 cerbB3 cerbB3 erbB3S p180ErbB3 p45sErbB3 p85sErbB3',
 'Pancreatic adenocarcinoma RB1 TP53 KRAS OSRC PPP1R130 RB p105Rb p110RB1 pRb pp110 BCC7 BMFS5 LFS1 P53 TRP53 CKRAS CKRAS CFC2 KRAS2A KRAS2B KRAS4A KRAS4B KRas KRas 2 KIRAS KRAS1 KRAS2 NS NS3 OES RALD RASK2 cKiras cKiras2',
 'Lung cancer EGFR L858R ERBB ERBB1 HER1 NISBD2 PIG61 mENA Malignant neoplasm lung of bronchus or lung NOS or lung cancer Malignant neoplasm of bronchus or lung NOS Mal

In [50]:
seq_b_test

['dMR During First Line Treatment of Non Squamous Lung Cancer: Time Course and Prognostic and Predictive Impact. To date, there are no methods to reliably select which patients with non-squamous non-small cell lung cancer (NSCLC) that benefit most from treatment with bevacizumab. Data have shown that high levels of plasma VEGF are prognostic and correlates with a worse disease outcome in some tumour types, including advanced NSCLC. Recent data are suggestive of a predictive value of imaging techniques for early detection of antiangiogenic treatment efficacy in different cancers. To our knowledge there are no presented data available on correlation between changes in diffusion-weighted MR and response to bevacizumab treatment in lung cancer. The current study is designed as a pilot study to prospectively investigate changes in MR variables during treatment with bevacizumab and to detect signals of prognostic and/or predictive value of MR changes during treatment.',
 'Efficacy and Safety

In [51]:
seq_a_test

['squamous cell lung cancer FGFR1 amplification BFGFR CD331 CEK ECCL FGFBR FGFR1 FLG FLT2 FLT2 HBGFR HH2 HRTFDS KAL2 NSAM OGD bFGFR1',
 'familial hypercholesterolemia PCSK9 FH3 FHCL3 HCHOLA3 LDLCQ1 NARC1 NARC1 PC9 proprotein convertase subtilisin kexin type 9 convertase subtilisin kexin type 9 preproprotein neural apoptosis regulated convertase 1 subtilisin kexinlike protease PC9 Familial hypercholesterolemia',
 'lung adenocarcinoma KRAS G12V high tumor mutational burden CKRAS CKRAS CFC2 KRAS2A KRAS2B KRAS4A KRAS4B KRas KRas 2 KIRAS KRAS1 KRAS2 NS NS3 OES RALD RASK2 cKiras cKiras2',
 'nonsmall cell lung cancer KRAS G12C CKRAS CKRAS CFC2 KRAS2A KRAS2B KRAS4A KRAS4B KRas KRas 2 KIRAS KRAS1 KRAS2 NS NS3 OES RALD RASK2 cKiras cKiras2 Nonsmall cell lung cancer Nonsmall cell lung cancer disorder NSCLC Nonsmall cell lung cancer',
 'bladder cancer FGFR3 S249C ACH CD333 CEK2 HSFGFR3EX JTK4 fibroblast growth factor receptor 3 FGFR3 fibroblast growth factor receptor 3 variant 4 fibroblast growth 

In [52]:
list(df_input[seq_a_col])

[['Pancreatic cancer',
  'CDK6 Amplification',
  'MCPH12 PLSTIRE',
  'cyclin-dependent kinase 6 cell division protein kinase 6 serine/threonine-protein kinase PLSTIRE',
  'Malignant tumor of pancreas',
  'Malignant tumor of pancreas (disorder) CA - Pancreatic cancer Pancreatic cancer CA - Cancer of pancreas Malignant tumour of pancreas'],
 ['Pancreatic adenocarcinoma',
  'KRAS, TP53',
  "'C-K-RAS C-K-RAS CFC2 K-RAS2A K-RAS2B K-RAS4A K-RAS4B K-Ras K-Ras 2 KI-RAS KRAS1 KRAS2 NS NS3 OES RALD RASK2 c-Ki-ras c-Ki-ras2 BCC7 BMFS5 LFS1 P53 TRP53"],
 ['Pancreatic ductal adenocarcinoma',
  'ERBB3',
  'ErbB-3 FERLK HER3 LCCS2 MDA-BF-1 c-erbB-3 c-erbB3 erbB3-S p180-ErbB3 p45-sErbB3 p85-sErbB3'],
 ['Pancreatic adenocarcinoma',
  'RB1, TP53, KRAS',
  "OSRC PPP1R130 RB p105-Rb p110-RB1 pRb pp110 BCC7 BMFS5 LFS1 P53 TRP53 'C-K-RAS C-K-RAS CFC2 K-RAS2A K-RAS2B K-RAS4A K-RAS4B K-Ras K-Ras 2 KI-RAS KRAS1 KRAS2 NS NS3 OES RALD RASK2 c-Ki-ras c-Ki-ras2"],
 ['Lung cancer',
  'EGFR (L858R)',
  'ERBB ERBB1 H

In [53]:
load_model_path = "../data/pretrained_models"
model_name = "scibert_scivocab_uncased"

num_labels = 2

model_path = f"{load_model_path}/{model_name}" # a path, osdir was set earlier (in Colab).
CONFIG = BertConfig.from_pretrained(f"{model_path}/{CONFIG_FILE}", num_labels=num_labels)
bert_dataloader = BertSeqClassDataHolder(
    model_path, CONFIG, num_labels, TOKEN_CLASS, vocab_file=VOCAB_FILE
)

In [54]:
batch_size = 8

bert_dataloader.load_pre_split_train_dataset(
    seq_a_train, seq_b_train, labels_train,
    None, None,
    batch_size=batch_size
)

SEQ_A_and_B: 27207it [00:06, 4434.24it/s]


In [55]:
tmp = bert_dataloader.X_train[500]
print(tmp[:150])
print(len(tmp))
tmp[-55:]

[102, 3183, 1314, 23867, 30132, 7462, 1389, 2642, 30131, 1750, 30132, 1750, 30132, 1541, 1750, 30132, 1965, 30111, 371, 1541, 3312, 30115, 10162, 30114, 30130, 7863, 27869, 3183, 131, 8645, 153, 234, 3183, 9594, 234, 3183, 1314, 7863, 27869, 131, 8645, 153, 234, 3183, 9594, 7863, 27869, 3183, 131, 8645, 153, 234, 3183, 9594, 234, 3183, 1314, 5087, 3183, 1314, 103, 9599, 9356, 389, 4344, 23663, 26650, 3304, 28522, 23365, 4344, 23663, 26650, 121, 23867, 30132, 145, 1750, 30132, 546, 1532, 9123, 3479, 1314, 238, 1937, 3321, 527, 165, 2933, 147, 3138, 9599, 9356, 389, 907, 191, 3970, 131, 2216, 10878, 121, 23867, 30132, 145, 1750, 30132, 546, 1532, 9123, 3479, 1314, 568, 4724, 147, 1979, 12026, 6820, 234, 27953, 19939, 30113, 205, 103, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
256


[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [ ]:
P_5                   	all	0.2621
P_10                  	all	0.1966
P_15                  	all	0.1655

year_set = [2017, 2018]

topic: 79
MODEL: clinical_bert
{'false_negative': 57,
 'false_positive': 9,
 'precision': 0.9,
 'r1ecall': 0.5869565217391305,
 'true_negative': 123,
 'true_positive': 81}
MODEL: clinical_bert_qe
{'false_negative': 137,
 'false_positive': 0,
 'precision': 1.0,
 'recall': 0.007246376811594203,
 'true_negative': 132,
 'true_positive': 1}
 
 topic: 115
MODEL: clinical_bert
{'false_negative': 22,
 'false_positive': 4,
 'precision': 0.2,
 'recall': 0.043478260869565216,
 'true_negative': 313,
 'true_positive': 1}
MODEL: clinical_bert_qe
{'false_negative': 23,
 'false_positive': 0,
 'precision': 0,
 'recall': 0.0,
 'true_negative': 317,
 'true_positive': 0}